# Meetup Cost Calculator

**Introduction:**

Organizing a successful technical meetup requires meticulous planning and attention to detail. One of the biggest challenges is ensuring that the event meets the needs and expectations of all participants, providing a valuable and memorable experience. In this blog, we will discuss how the integration between Elasticsearch, NVIDIA-NIM, and generative AI can assist in organizing a flawless technical meetup, from gathering information to generating personalized reports.

In [ ]:
!pip install langchain_nvidia_ai_endpoints
!pip install langchain-community langchain-text-splitters langchain
!pip install -q -U langchain-elasticsearch python-dotenv

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_elasticsearch import ElasticsearchStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from urllib.request import urlopen
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

# Replace 'path/to/your/.env' with the correct path to your .env file on Google Drive
env_path = '/content/drive/MyDrive/@Blogs/05-NVIDIA-NIM/env_'
load_dotenv(env_path)

# Elastic cloud credentials
es_cloud_id = os.getenv('cloud_id')
es_user = os.getenv('cloud_user')
es_pass = os.getenv('cloud_pass')

ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')

elastic_index_name = "01-rio-meetup"

In [ ]:
# os.environ['NVIDIA_API_KEY'] = "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"
os.environ['NVIDIA_API_KEY'] = os.getenv('NVIDIA_API_KEY')


In [ ]:
# Download the dataset

url = "https://raw.githubusercontent.com/salgado/public-dataset/main/meetup-participants.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

In [ ]:
# Split Documents into Passages
metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["observation"])
    metadata.append(
        {
            "name": doc["name"],
            "observation": doc["observation"]
        }
    )

text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
docs = text_splitter.create_documents(content, metadatas=metadata)

In [ ]:
query_embedding = NVIDIAEmbeddings()

es = ElasticsearchStore.from_documents(
    docs,
    es_cloud_id=es_cloud_id,
    es_api_key=ELASTIC_API_KEY,
    index_name=elastic_index_name,
    embedding=query_embedding,
)

In [ ]:
retriever = es.as_retriever(search_kwargs={"k": 20})

In [ ]:
model = ChatNVIDIA(model="mistral_7b")

In [ ]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [ ]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [ ]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [ ]:
for s in final_chain.stream("How many guests are vegetarian?"):
    print(s, end="")

Based on the provided context, there are a total of 7 guests with vegetarian restrictions. Their names are João Silva, Julia Carvalho, Fernanda Lima, Tiago Mendes, Ana Costa, and Patricia Rocha.

In [ ]:
prompt = """Generate a menu of food and drink to be served, respecting dietary restrictions.\n

Generate a single order that can be placed on a food and drink delivery website.
We will buy spicy sausage pizza, vegan pizza, and gluten-free Margherita pizza.
For drinks, we will have Coca-Cola, water, beer, and grape juice.
Calculate the quantity of the order so that each guest can eat 3 slices of pizza and drink 500ml of beverage.
1 pizza contains 8 slices.

Finally, Estimate the cost of this purchase based on average prices in the city of Rio de Janeiro.

"""

for s in final_chain.stream(prompt):
    print(s, end="")

To create a menu that respects the dietary restrictions of the guests mentioned in the context, we will order the following:

1. Spicy sausage pizza (for guests with no dietary restrictions): 1 pizza (8 slices)
2. Vegan pizza (for guests with vegan restrictions): 1 pizza (8 slices)
3. Gluten-free Margherita pizza (for guests with gluten-free restrictions): 1 pizza (8 slices)

For drinks, we will have:
1. Coca-Cola (for guests with no dietary restrictions): 3 bottles (330ml each)
2. Water (for guests with celiac, lactose intolerant, and diabetic restrictions): 7 bottles (300ml each)
3. Beer (for guests with no dietary restrictions): 2 cans (355ml each)
4. Grape juice (for guests with vegetarian, sugar-free, and vegan restrictions): 3 bottles (300ml each)

To ensure that each guest can eat 3 slices of pizza and drink 300ml of beverage, we need to calculate the total number of slices and bottles required:

Total number of guests: 15
Total number of slices needed: 15 * 3 = 45 slices
Total 